In [1]:
import copy
import pickle

import pathlib as pl
import pandas as pd
import networkx as nx

from justhink_world import create_all_worlds
from justhink_world.domain.state import NetworkState, EnvironmentState
from justhink_world.agent import Human, Robot
from justhink_world.domain.action import *

In [2]:
pd.set_option('display.max_rows', 500)

action_type_dict = {
    'TYPE_SUGGEST_PICK': 0,
    'TYPE_PICK': 1,
    'TYPE_UNPICK': 2,
    'TYPE_SUBMIT': 3,
    'TYPE_SUGGEST_SUBMIT': 4,
    'TYPE_CLEAR_SUGGEST_SUBMIT': 5,
    'TYPE_AGREE': 6,
    'TYPE_DISAGREE': 7,
    'TYPE_CLEAR': 8,
    'TYPE_GUESS': 9}

# Define paths.

In [3]:
resources_dir = pl.Path('resources')
networks_dir = resources_dir.joinpath('networks')
tables_dir = pl.Path('../processed_data/log_tables')

processed_tables_pickle_file = pl.Path(
    '../processed_data/processed_tables.pickle')
transitions_pickle_file = pl.Path('../processed_data/justhink_spring21_transition_logs.pickle')

# Initializing Problems.

In [4]:
worlds = create_all_worlds()

In [5]:
name = 'pretest-1'
name = 'collab-activity-1'

worlds[name].env.state.network.graph.edges(data=True), worlds[name].env.state.network.subgraph.edges(data=True)

(EdgeDataView([(3, 1, {'cost': 3}), (3, 0, {'cost': 3}), (3, 2, {'cost': 3}), (3, 5, {'cost': 4}), (3, 7, {'cost': 5}), (1, 9, {'cost': 3}), (1, 2, {'cost': 2}), (1, 5, {'cost': 4}), (1, 4, {'cost': 3}), (0, 2, {'cost': 5}), (0, 7, {'cost': 4}), (2, 9, {'cost': 4}), (2, 8, {'cost': 3}), (5, 4, {'cost': 2}), (5, 7, {'cost': 3}), (5, 6, {'cost': 4}), (4, 9, {'cost': 3}), (4, 6, {'cost': 2}), (7, 6, {'cost': 2}), (9, 8, {'cost': 2})]),
 EdgeDataView([]))

In [6]:
world_renamer = {name: name for name in worlds} 
world_renamer['indiv-illustrate'] = 'demo'
world_renamer['collab-activity'] = 'collab-activity-1'

# Make dictionaries for dataframe traversal.

In [7]:
# dictionary for file path, key is student id
file_list = tables_dir.glob(
    'justhink20_log_*/justhink_app-state_transition.csv')
files = {}

for f in file_list:
    i = int(str(f.parent).split('_')[-1])
    files[i] = f

# dictionary for the state transition tables, key is student id
tables = {}

for student in files:
    df = pd.read_csv(files[student])
    tables[student] = df

In [8]:
tables[1].columns

Index(['Time', 'header.seq', 'header.frame_id', 'state.edges',
       'state.suggested.u', 'state.suggested.v', 'state.terminal',
       'state.submit_suggested', 'action.agent_name', 'action.type',
       'action.edge.u', 'action.edge.v', 'next_state.edges',
       'next_state.suggested.u', 'next_state.suggested.v',
       'next_state.terminal', 'next_state.submit_suggested', 'action_no',
       'step_no', 'turn_agent'],
      dtype='object')

# Reconstruct a State.

In [9]:
# reconstructing the state of the input row
def reconstruct_state(row, graph, state_name='state', verbose=False):
    # row_index refers to table row +2
    #     row = full_row[3:9]
    #     print(row)

    # reconstructs edge list from string
    edge_list_text = str(row['{}.edges'.format(state_name)])
    edge_list = edge_list_text.strip('[').strip(']').split(',')
    edges = list()
    for edge_text in edge_list:
        # if the edge list is empty
        if edge_list[0] == '':
            break
        l = edge_text.split()
        edge = (int(l[1]), int(l[3]))
        edges.append(edge)
    # edges = frozenset(edges)
    subgraph = nx.Graph()
    for u, v in edges:
        subgraph.add_edge(u, v)

    suggested_edge = (int(row['{}.suggested.u'.format(state_name)]),
                      int(row['{}.suggested.v'.format(state_name)]))
    if suggested_edge == (-1, -1):
        suggested_edge = None

    is_terminal = bool(row['{}.terminal'.format(state_name)])
    is_submitting = bool(row['{}.submit_suggested'.format(state_name)])

    if row['turn_agent'] == 'human':
        agents = frozenset({Human})
    elif row['turn_agent'] == 'robot':
        agents = frozenset({Robot})
    else:
        agents = frozenset()

    network = NetworkState(graph, subgraph, suggested_edge)
    state = EnvironmentState(
        network, agents=agents, is_submitting=is_submitting, is_terminal=is_terminal)

    if verbose:
        print(state)
    return state

In [10]:
world = worlds['pretest-1']
df = tables[1].copy()
row_index = 10
reconstruct_state(df.iloc[row_index], world.env.state.network.graph, verbose=True)

EnvironmentState(NetworkState(e:6+0,c:19|n:7,e:12;s:0)@1/inf,a:H;p:0,t:0,s:0,h:0)


EnvironmentState(NetworkState(e:6+0,c:19|n:7,e:12;s:0)@1/inf,a:H;p:0,t:0,s:0,h:0)

# Reconstruct an Action object.

In [11]:
# reconstructing action objects by action type
def reconstruct_action(row, graph, verbose=False):
    action_type = row['action.type']
    if row['action.agent_name'] == 'human':
        agent = Human
    elif row['action.agent_name'] == 'robot':
        agent = Robot
    else:
        raise NotImplementedError
        
    if action_type == 0:
        return SuggestPickAction((row['action.edge.u'], row['action.edge.v']), agent)
    if action_type == 1:
        return PickAction((row['action.edge.u'], row['action.edge.v']), agent)
    if action_type == 2:
        return UnpickAction((row['action.edge.u'], row['action.edge.v']), agent)
    if action_type == 3:
        return SubmitAction(agent)
    if action_type == 4:
        # return SuggestSubmitAction(agent)
        return AttemptSubmitAction(agent)
    if action_type == 5:
        # return ClearSuggestSubmitAction(agent)
        return ContinueAction(agent)
    if action_type == 6:
        return AgreeAction(agent)
    if action_type == 7:
        return DisagreeAction(agent)
    if action_type == 8:
        return ClearAction()
    if action_type == 9:
        print("TYPE_GUESS")
    return None

In [12]:
world = worlds['pretest-1']
df = tables[1].copy()
row_index = 10
reconstruct_action(df.iloc[row_index], world.env.state.network.graph, verbose=True)

Action(pick(5,6),Human)

# Reconstruct all States and Action objects.

Appending world_state, action, is_submission, cost, and is_mst columns to the table.

In [13]:
# reconstruct the state for every row in the table
def construct_state_from_table(df, state_name='state', verbose=False):
    state_list = []
    is_submission = []
    cost = []
    is_mst = []
    action_list = []

    attempt_nos = dict()
    for i, row in df.iterrows():
        problem_name = row['header.frame_id']
        # Renamed scenes.
        problem_name = world_renamer[problem_name]
        problem = worlds[problem_name]
        
        if problem_name not in attempt_nos:
            attempt_nos[problem_name] = 1
        
        action = reconstruct_action(row, problem.env.state.network.graph)
        action_list.append(action)

        
        state = reconstruct_state(
            row, problem.env.state.network.graph, state_name=state_name)

        submission = identify_submission_state(row)
        is_submission.append(submission)
        if verbose and submission:
            print('Submitted', problem_name, state)

        # Update the attempt number.
        if submission:
            attempt_nos[problem_name] = attempt_nos[problem_name] + 1
        state.attempt_no = attempt_nos[problem_name]

        # Update the max attempts.
        if 'collab-activity' in problem_name:
            state.max_attempts = 4
            
        state_list.append(state)

        cost.append(state.network.get_cost())
        is_mst.append(state.network.is_mst())

    # append worldstate column
    df[state_name] = state_list
    # append action object column
    df['action'] = action_list
    # append is submission column
    df['is_submission'] = is_submission
    # append cost column
    df['cost'] = cost
    # append is_mst column
    df['is_mst'] = is_mst

    return df

### Identifying submission state.

In [14]:
# identifies which problem state has been submitted
def identify_submission_state(row):
    # check if the last state's action type is submit type 3 or 4
    problem_name = row['header.frame_id']
    
    # Renamed scenes.
    problem_name = world_renamer[problem_name]
        
    action_type = row['action.type']

    if 'test' in problem_name:
        return action_type == 4

    # for collaborative activities check for 3 only
    elif "collab" in problem_name:
        return action_type == 3

    else:
        return False


In [15]:
name = 'collab-activity'
name = 'pretest-1'
df = tables[1].copy()
df = construct_state_from_table(df, state_name='next_state')
df = construct_state_from_table(df, verbose=True)


pd.options.display.max_columns = None
df

Submitted pretest-1 EnvironmentState(NetworkState(e:7+0,c:23|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:0,h:0)
Submitted pretest-1 EnvironmentState(NetworkState(e:7+0,c:23|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:1,h:0)
Submitted pretest-2 EnvironmentState(NetworkState(e:7+0,c:38|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:0,h:0)
Submitted pretest-2 EnvironmentState(NetworkState(e:7+0,c:38|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:1,h:0)
Submitted pretest-3 EnvironmentState(NetworkState(e:6+0,c:22|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:0,h:0)
Submitted pretest-3 EnvironmentState(NetworkState(e:6+0,c:22|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:1,h:0)
Submitted pretest-4 EnvironmentState(NetworkState(e:6+0,c:45|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:0,h:0)
Submitted pretest-4 EnvironmentState(NetworkState(e:6+0,c:45|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:1,h:0)
Submitted pretest-5 EnvironmentState(NetworkState(e:6+0,c:15|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:0,h:0)
Submitted pretest-5 EnvironmentState(NetworkState(e:6+0,c:15|n:7,e:12;s:1)@1/inf,a

,Time,header.seq,header.frame_id,state.edges,state.suggested.u,state.suggested.v,state.terminal,state.submit_suggested,action.agent_name,action.type,action.edge.u,action.edge.v,next_state.edges,next_state.suggested.u,next_state.suggested.v,next_state.terminal,next_state.submit_suggested,action_no,step_no,turn_agent,next_state,action,is_submission,cost,is_mst,state
0,0.000000,1,indiv-illustrate,[],-1,-1,False,False,human,1,1,9,[u: 1\nv: 9],-1,-1,False,False,0,1,human,"EnvironmentState(NetworkState(e:1+0,c:3|n:2,e:...","Action(pick(1,9),Human)",False,0,False,"EnvironmentState(NetworkState(e:0+0,c:0|n:2,e:..."
1,36.886868,2,indiv-illustrate,[u: 1\nv: 9],-1,-1,False,False,human,8,-1,-1,[],-1,-1,False,False,1,2,human,"EnvironmentState(NetworkState(e:0+0,c:0|n:2,e:...","Action(clear,Human)",False,3,True,"EnvironmentState(NetworkState(e:1+0,c:3|n:2,e:..."
2,54.157379,3,indiv-illustrate,[],-1,-1,False,False,human,1,1,9,[u: 1\nv: 9],-1,-1,False,False,2,3,human,"EnvironmentState(NetworkState(e:1+0,c:3|n:2,e:...","Action(pick(1,9),Human)",False,0,False,"EnvironmentState(NetworkState(e:0+0,c:0|n:2,e:..."
3,56.384898,4,indiv-illustrate,[u: 1\nv: 9],-1,-1,False,False,human,3,-1,-1,[u: 1\nv: 9],-1,-1,True,False,3,4,human,"EnvironmentState(NetworkState(e:1+0,c:3|n:2,e:...","Action(submit,Human)",False,3,True,"EnvironmentState(NetworkState(e:1+0,c:3|n:2,e:..."
4,96.414340,5,pretest-1,[],-1,-1,False,False,human,1,7,0,[u: 7\nv: 0],-1,-1,False,False,0,0,human,"EnvironmentState(NetworkState(e:1+0,c:4|n:7,e:...","Action(pick(7,0),Human)",False,0,False,"EnvironmentState(NetworkState(e:0+0,c:0|n:7,e:..."
5,101.380167,6,pretest-1,[u: 7\nv: 0],-1,-1,False,False,human,1,0,3,"[u: 7\nv: 0, u: 0\nv: 3]",-1,-1,False,False,1,0,human,"EnvironmentState(NetworkState(e:2+0,c:7|n:7,e:...","Action(pick(0,3),Human)",False,4,False,"EnvironmentState(NetworkState(e:1+0,c:4|n:7,e:..."
6,107.648563,7,pretest-1,"[u: 7\nv: 0, u: 0\nv: 3]",-1,-1,False,False,human,1,3,5,"[u: 7\nv: 0, u: 3\nv: 5, u: 0\nv: 3]",-1,-1,False,False,2,0,human,"EnvironmentState(NetworkState(e:3+0,c:11|n:7,e...","Action(pick(3,5),Human)",False,7,False,"EnvironmentState(NetworkState(e:2+0,c:7|n:7,e:..."
7,110.681328,8,pretest-1,"[u: 7\nv: 0, u: 3\nv: 5, u: 0\nv: 3]",-1,-1,False,False,human,1,7,5,"[u: 7\nv: 5, u: 7\nv: 0, u: 3\nv: 5, u: 0\nv: 3]",-1,-1,False,False,3,0,human,"EnvironmentState(NetworkState(e:4+0,c:14|n:7,e...","Action(pick(7,5),Human)",False,11,False,"EnvironmentState(NetworkState(e:3+0,c:11|n:7,e..."
8,119.594035,9,pretest-1,"[u: 7\nv: 5, u: 7\nv: 0, u: 3\nv: 5, u: 0\nv: 3]",-1,-1,False,False,human,1,5,4,"[u: 0\nv: 3, u: 7\nv: 5, u: 7\nv: 0, u: 5\nv: ...",-1,-1,False,False,4,0,human,"EnvironmentState(NetworkState(e:5+0,c:16|n:7,e...","Action(pick(5,4),Human)",False,14,False,"EnvironmentState(NetworkState(e:4+0,c:14|n:7,e..."
9,122.760478,10,pretest-1,"[u: 0\nv: 3, u: 7\nv: 5, u: 7\nv: 0, u: 5\nv: ...",-1,-1,False,False,human,1,4,1,"[u: 7\nv: 5, u: 5\nv: 4, u: 0\nv: 3, u: 4\nv: ...",-1,-1,False,False,5,0,human,"EnvironmentState(NetworkState(e:6+0,c:19|n:7,e...","Action(pick(4,1),Human)",False,16,False,"EnvironmentState(NetworkState(e:5+0,c:16|n:7,e..."


In [16]:
for index,row in df.iterrows():
    break

# Storing processed tables into a dictionary

In [17]:
processed_tables = {}
for key, df in tables.items():
    df = df.copy()
    df = construct_state_from_table(df, state_name='next_state')
    df = construct_state_from_table(df, state_name='state')

    processed_tables[key] = df
    print('Processed', key)

Processed 7
Processed 3
Processed 2
Processed 6
Processed 5
Processed 4
Processed 1
Processed 9
Processed 10


# Export processed tables

In [18]:
# with processed_tables_pickle_file.open('wb') as handle:
#     pickle.dump(processed_tables, handle, protocol=pickle.HIGHEST_PROTOCOL)

# print('Saved processed tables to {}'.format(processed_tables_pickle_file))

# Construct state transition lists per world.

In [19]:
transitions = dict()

for student_no, log_df in processed_tables.items():
    transitions[student_no] = dict()

    for name in world_renamer:

        df = log_df[log_df['header.frame_id'] == name].reset_index()

        trans_list = []
        for i, row in df.iterrows():
            if i == 0:
                trans_list.append(row['state'])
            trans_list.append(row['action'])
            trans_list.append(row['next_state'])

        if len(trans_list) != 0:
            print('Added {} states for {} at {}'.format(
                len(trans_list)+1//2, student_no, name))
            transitions[student_no][world_renamer[name]] = trans_list

Added 41 states for 7 at pretest-1
Added 17 states for 7 at pretest-2
Added 13 states for 7 at pretest-3
Added 9 states for 7 at pretest-4
Added 11 states for 7 at pretest-5
Added 11 states for 7 at posttest-1
Added 7 states for 7 at posttest-2
Added 9 states for 7 at posttest-3
Added 7 states for 7 at posttest-4
Added 11 states for 7 at posttest-5
Added 175 states for 7 at collab-activity-2
Added 9 states for 7 at indiv-illustrate
Added 95 states for 7 at collab-activity
Added 17 states for 3 at pretest-1
Added 17 states for 3 at pretest-2
Added 17 states for 3 at pretest-3
Added 31 states for 3 at pretest-4
Added 17 states for 3 at pretest-5
Added 17 states for 3 at posttest-1
Added 17 states for 3 at posttest-2
Added 17 states for 3 at posttest-3
Added 17 states for 3 at posttest-4
Added 17 states for 3 at posttest-5
Added 199 states for 3 at collab-activity-2
Added 9 states for 3 at indiv-illustrate
Added 163 states for 3 at collab-activity
Added 7 states for 2 at pretest-1
Added 3

# Export transition lists

In [20]:
transitions[1]['pretest-1']

[EnvironmentState(NetworkState(e:0+0,c:0|n:7,e:12;s:0)@1/inf,a:H;p:0,t:0,s:0,h:0),
 Action(pick(7,0),Human),
 EnvironmentState(NetworkState(e:1+0,c:4|n:7,e:12;s:0)@1/inf,a:H;p:0,t:0,s:0,h:0),
 Action(pick(0,3),Human),
 EnvironmentState(NetworkState(e:2+0,c:7|n:7,e:12;s:0)@1/inf,a:H;p:0,t:0,s:0,h:0),
 Action(pick(3,5),Human),
 EnvironmentState(NetworkState(e:3+0,c:11|n:7,e:12;s:0)@1/inf,a:H;p:0,t:0,s:0,h:0),
 Action(pick(7,5),Human),
 EnvironmentState(NetworkState(e:4+0,c:14|n:7,e:12;s:0)@1/inf,a:H;p:0,t:0,s:0,h:0),
 Action(pick(5,4),Human),
 EnvironmentState(NetworkState(e:5+0,c:16|n:7,e:12;s:0)@1/inf,a:H;p:0,t:0,s:0,h:0),
 Action(pick(4,1),Human),
 EnvironmentState(NetworkState(e:6+0,c:19|n:7,e:12;s:0)@1/inf,a:H;p:0,t:0,s:0,h:0),
 Action(pick(5,6),Human),
 EnvironmentState(NetworkState(e:7+0,c:23|n:7,e:12;s:1)@1/inf,a:H;p:0,t:0,s:0,h:0),
 Action(attempt-submit,Human),
 EnvironmentState(NetworkState(e:7+0,c:23|n:7,e:12;s:1)@2/inf,a:H;p:0,t:0,s:1,h:0),
 Action(attempt-submit,Human),
 En

In [21]:
with transitions_pickle_file.open('wb') as handle:
    pickle.dump(transitions, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('Saved transition lists to {}'.format(transitions_pickle_file))

Saved transition lists to ../processed_data/justhink_spring21_transition_logs.pickle
